In [1]:
#Digital footprint for a given data group

In [52]:
import pandas as pd
from collections import Counter

In [53]:
import nltk
from nltk.corpus import stopwords # Import the stop word list

In [4]:
df = pd.read_csv('../data/dsi/nutch-crawl-01-14-2017-cleaned.csv')

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87458 entries, 0 to 87457
Data columns (total 7 columns):
Unnamed: 0       87458 non-null int64
tstamp           87458 non-null object
domain           87458 non-null object
url              87458 non-null object
content          87458 non-null object
title            86449 non-null object
clean_content    87458 non-null object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [101]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np

In [40]:
#remove non letters content
import re
df['clean_content'] = df.content.map(lambda x: re.sub('[^a-zA-Z]+', '', x))

In [55]:
words = df['content'].str.split()

In [103]:
stemmer = nltk.stem.porter.PorterStemmer()
def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

In [104]:
'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

In [65]:
flatten = [re.sub('[^a-zA-Z]+', '', item) for sublist in words for item in sublist if len(re.sub('[^a-zA-Z]+', '', item)) != 0]

flatten = [item for sublist in words for item in sublist]

In [66]:
count = Counter(flatten)

In [112]:
top_200 = count.most_common(200)

In [113]:
for t in top_200:
    print t[0], t[1]

the 1742649
and 1496076
to 1302550
of 1249910
a 858610
in 761454
for 603562
Data 586174
is 505719
on 406458
data 363768
The 356056
with 350901
that 331389
by 274638
you 262808
are 255576
Analytics 240636
or 236004
your 217707
as 211291
be 210936
at 208962
Big 197172
from 196602
this 182396
an 170320
can 161787
it 160143
Science 154560
more 153787
will 136042
have 129059
In 122711
I 121003
not 114389
News 109897
our 107832
A 104972
Business 99927
we 98790
Learning 95317
has 94883
about 94160
This 93348
About 92795
all 91933
their 91883
new 89484
Contact 88299
business 84930
analytics 84849
use 83383
More 81414
All 80988
information 78426
Management 78223
its 76392
Blog 76008
Us 75573
was 74857
how 73229
which 72623
Search 72188
Home 72177
Twitter 70339
How 68418
For 68140
You 66772
New 66356
but 66280
University 65771
one 65479
other 65456
Research 65006
any 62987
IBM 62959
they 61926
Events 61875
Machine 61852
We 60905
up 60843
Top 60163
also 59325
time 59141
R 58935
Services 58339
Had

In [111]:
for word, cnt in count.iteritems():
    if cnt > 500:
        print word, cnt

Euro 563
College 15024
Retention 1050
fit 4245
fix 2542
Sitemap 4017
Meetups 10853
discipline 1324
Setup 513
Framed 770
Frames 508
IBMAnalytics 732
Calls 1051
chain 2736
chair 1227
exact 1569
Proposals 1236
CONTENT 621
Significant 747
sorted 550
Competitions 1660
Worldwide 3122
sponsorship 656
Harbor 760
article 15850
readings 533
talented 996
Size 1998
Clear 1410
IRM 723
Clean 2865
COO 642
with 350901
hash 557
automate 1298
TransCAD 690
learn 18879
Authority 795
Cars 1530
Cart 2203
Carl 925
Card 2852
Care 4700
influence 2451
evolutionary 783
could 21648
Weather 3380
Rating 948
SanDisk 1440
become 12410
Speaking 1522
Myanmar 525
iitalic 721
U 5121
integrity 1371
replication 815
carriers 582
innovative 5552
production 5598
Motion 1072
Type 9497
illustrate 871
contents 1793
Button 1016
QA 2003
runs 2612
night 1750
Italia 816
Displays 582
Form 3727
Ford 1459
Webinar 13088
Drivers 1121
dependencies 972
advantages 1725
transition 1485
transferred 605
Sharing 1434
GNU 1967
metric 1653
exciti

In [109]:
for i in range(1000, 3000):
    v[i], n[i]

TypeError: unhashable type

In [67]:
print len(count)

792505


In [105]:
corpus = [ word for (word, cnt) in count.iteritems() if cnt!=1 and word not in stopwords.words('english')]
#corpus = [word for (word, cnt) in count.iteritems() if cnt!=1 and not word.isdigit()]

In [106]:
print len(corpus) #with stop words (370487) and no stop words (370360)

370360


In [81]:
vectorizer = TfidfVectorizer(stop_words='english')
def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [91]:
msk = np.random.rand(len(df)) < 0.001
train = df[msk]
test = df[~msk]

In [92]:
train_words = train['content'].str.split()
test_words = test['content'].str.split()

In [93]:
train_flatten = [re.sub('[^a-zA-Z]+', '', item) for sublist in train_words for item in sublist if len(re.sub('[^a-zA-Z]+', '', item)) != 0]
test_flatten = [re.sub('[^a-zA-Z]+', '', item) for sublist in test_words for item in sublist if len(re.sub('[^a-zA-Z]+', '', item)) != 0]

In [94]:
train_count = Counter(train_flatten)
test_count = Counter(test_flatten)

In [107]:
train_corpus = [ word for (word, cnt) in train_count.iteritems() if cnt!=1 and word not in stopwords.words('english')]
test_corpus = [ word for (word, cnt) in test_count.iteritems() if cnt!=1 and word not in stopwords.words('english')]

In [96]:
text1 = ' '.join(train_corpus)
text2 = ' '.join(test_corpus)
print cosine_sim(text1, text2)

0.174632169848


In [97]:
train = ' '.join(corpus)

In [99]:
#for 4:6 split: text1:0.8321, text2: 0.7321 
#for 1:999 split: text1: 
print cosine_sim(train, text2)  

0.999451547437


In [100]:
print len(train), len(text2)

3827955 3824475
